# Corrective Newtork

A corrective network, also known as a compensator or controller, is a component added to a control system to modify its performance and improve its stability, transient response, and steady-state error. The primary goal of a corrective network is to achieve desired system behavior or specifications in the presence of disturbances, uncertainties, and variations.

# Lead Controller

- Advantage: Increase Phase Margin around ωm enhancing system stability, improves settling times and reduces overshoot for a faster and smoother response
- Disadvantage: Increase gain for ω > 1/τ

<br>
<img src="../images/lead.png" alt="Lead Controller" width="40%" height="40%">



In [56]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import ipywidgets as widgets
from ipywidgets import interact
from IPython.display import display, clear_output

def plot_lead_controller(alpha, tau):
    # Define the lead compensator transfer function
    num_lead = [tau, 1]
    den_lead = [alpha * tau, 1]
    lead_sys = signal.TransferFunction(num_lead, den_lead)

    # Frequency range for Bode plot
    freq_range = np.logspace(-2, 2, 1000)
    _, magnitude, phase = signal.bode(lead_sys, freq_range)

    # Clear previous plot
    clear_output(wait=True)

    # Plot Bode diagram
    plt.figure(figsize=(10, 6))

    # Magnitude plot
    plt.subplot(2, 1, 1)
    plt.semilogx(freq_range, magnitude)
    plt.title('Bode Diagram')
    plt.xlabel('Frequency (rad/s)')
    plt.ylabel('Magnitude (dB)')
    plt.ylim(-10, 30)
    plt.grid(True)

    # Peak value
    peak_index = np.argmax(magnitude)
    peak_magnitude = magnitude[peak_index]

    # Draw the tau influence
    plt.plot([1e-2, 1/tau], [0, 0], linestyle='--', color='green', label='τ break')

    # Draw the alpha * tau influence
    plt.plot([1/(alpha * tau), 1e2], [peak_magnitude, peak_magnitude], linestyle='--', color='lightgreen', label='τ•α break')

    # Draw the connection
    plt.plot([1/tau, 1/(alpha * tau)], [0, peak_magnitude], linestyle='--', color='gray')

    plt.legend()

    # Phase plot
    plt.subplot(2, 1, 2)
    plt.semilogx(freq_range, phase * np.pi/180)
    plt.xlabel('Frequency (rad/s)')
    plt.ylabel('Phase (rad)')

    # Mean pulse
    wm = 1/(tau * np.sqrt(alpha))

    plt.plot([wm, wm], [-np.pi, np.pi], linestyle='--', color='blue', label='ωm')
    
    # Set custom yticks for phase between pi and -pi
    plt.yticks(np.arange(-np.pi, np.pi + np.pi/2, np.pi/2),
               ['-π', '-π/2', '0', 'π/2', 'π'])
    
    plt.grid(True)
    plt.tight_layout()
    plt.legend()
    plt.show()

# Create sliders for alpha and tau
interact(plot_lead_controller,
         alpha=widgets.FloatSlider(min=0.1, max=1, step=0.1, value=0.0, description='Alpha'),
         tau=widgets.FloatSlider(min=0.1, max=10, step=0.1, value=1.0, description='Tau'))


interactive(children=(FloatSlider(value=0.1, description='Alpha', max=1.0, min=0.1), FloatSlider(value=1.0, de…

<function __main__.plot_lead_controller(alpha, tau)>

## Lead controller effect on closed loop

In [52]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact
import control as ctrl

# Function to plot step response and Bode diagram
def plot_feedback_system(pole1, pole2, zero, gain, alpha, tau):
    # Define the transfer function G(s) = K*(s - zero) / [(s - pole1)*(s - pole2)]
    G = ctrl.TransferFunction([gain, -gain * zero], [1, -(pole1 + pole2), pole1 * pole2])

    # Define the controller R(s) = (1 + s*tau) / (1 + s*alpha*tau)
    R = ctrl.TransferFunction([1, tau], [1, alpha * tau])

    # Calculate the feedback system T(s) = (R*G) / (1 + R*G)
    T = ctrl.feedback(R * G, 1)

    # Time vector for step response
    time, response = ctrl.step_response(T)

    # Frequency range for Bode plot
    freq_range = np.logspace(-2, 2, 1000)
    _, magnitude, phase = signal.bode(signal.TransferFunction(T.num[0][0], T.den[0][0]), freq_range)
    #_, magnitude, phase = signal.bode(signal.TransferFunction(np.array(T.num), np.array(T.den)), freq_range)

    # Plot step response
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 2)
    plt.plot(time, response)
    plt.title('Step Response of closed loop')
    plt.xlabel('Time')
    plt.ylabel('Amplitude')
    plt.grid(True)

    # Plot Bode diagram

    # Magnitude plot
    plt.subplot(2, 2, 1)
    plt.semilogx(freq_range, magnitude)
    plt.title('Bode Diagram of closed loop')
    plt.xlabel('Frequency (rad/s)')
    plt.ylabel('Magnitude (dB)')
    plt.ylim(-30, 10)
    plt.grid(True)

    # Phase plot
    plt.subplot(2, 2, 3)
    plt.semilogx(freq_range, phase * np.pi/180)
    plt.xlabel('Frequency (rad/s)')
    plt.ylabel('Phase (rad)')
    
    # Set custom yticks for phase between pi and -pi
    plt.yticks(np.arange(-np.pi, np.pi + np.pi/2, np.pi/2),
               ['-π', '-π/2', '0', 'π/2', 'π'])
    
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    # Pole-zero map
    plt.figure(figsize=(12, 4))
    poles = G.pole()
    zeros = G.zero()
    plt.scatter(np.real(poles), np.imag(poles), marker='x', color='red', label='Poles')
    plt.scatter(np.real(zeros), np.imag(zeros), marker='o', color='blue', label='Zeros')
    plt.title('Pole-Zero Map of G(s)')
    plt.xlabel('Real')
    plt.ylabel('Imaginary')
    plt.axhline(0, color='black',linewidth=0.5)
    plt.axvline(0, color='black',linewidth=0.5)
    plt.grid(True)
    plt.legend()

# Create sliders for poles, zero, gain, alpha, and tau
pole1_slider = widgets.FloatSlider(min=-5, max=5, step=0.1, value=-1, description='Pole 1:')
pole2_slider = widgets.FloatSlider(min=-5, max=5, step=0.1, value=-2, description='Pole 2:')
zero_slider = widgets.FloatSlider(min=-5, max=5, step=0.1, value=1, description='Zero:')
gain_slider = widgets.FloatSlider(min=0.1, max=10, step=0.1, value=1, description='Gain:')
alpha_slider = widgets.FloatSlider(min=0, max=1, step=0.1, value=1, description='Alpha:')
tau_slider = widgets.FloatSlider(min=0.1, max=10, step=0.1, value=1, description='Tau:')

# Create interactive plot
interact(plot_feedback_system,
         pole1=pole1_slider, pole2=pole2_slider,
         zero=zero_slider, gain=gain_slider,
         alpha=alpha_slider, tau=tau_slider)


interactive(children=(FloatSlider(value=-1.0, description='Pole 1:', max=5.0, min=-5.0), FloatSlider(value=-2.…

<function __main__.plot_feedback_system(pole1, pole2, zero, gain, alpha, tau)>

# Delay Controller

- Reduce gain for  ω > 1/τ
- Reduce Phase Margin around  ω and time delays can lead to instability in control systems

<br>
<img src="../images/delay.png" alt="Delay Controller" width="40%" height="40%">



In [58]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import ipywidgets as widgets
from ipywidgets import interact
from IPython.display import display, clear_output

def plot_lead_controller(alpha, tau):
    # Define the lead compensator transfer function
    num_lead = [alpha * tau, 1]
    den_lead = [tau, 1]
    lead_sys = signal.TransferFunction(num_lead, den_lead)

    # Frequency range for Bode plot
    freq_range = np.logspace(-2, 2, 1000)
    _, magnitude, phase = signal.bode(lead_sys, freq_range)

    # Clear previous plot
    clear_output(wait=True)

    # Plot Bode diagram
    plt.figure(figsize=(10, 6))

    # Magnitude plot
    plt.subplot(2, 1, 1)
    plt.semilogx(freq_range, magnitude)
    plt.title('Bode Diagram')
    plt.xlabel('Frequency (rad/s)')
    plt.ylabel('Magnitude (dB)')
    plt.ylim(-30, 10)
    plt.grid(True)

    # Peak value
    peak_index = np.argmin(magnitude)
    peak_magnitude = magnitude[peak_index]

    # Draw the tau influence
    plt.plot([1e-2, 1/tau], [0, 0], linestyle='--', color='green', label='τ break')

    # Draw the alpha * tau influence
    plt.plot([1/(alpha * tau), 1e2], [peak_magnitude, peak_magnitude], linestyle='--', color='lightgreen', label='τ•α break')

    # Draw the connection
    plt.plot([1/tau, 1/(alpha * tau)], [0, peak_magnitude], linestyle='--', color='gray')

    plt.legend()

    # Phase plot
    plt.subplot(2, 1, 2)
    plt.semilogx(freq_range, phase * np.pi/180)
    plt.xlabel('Frequency (rad/s)')
    plt.ylabel('Phase (rad)')

    # Mean pulse
    wm = 1/(tau * np.sqrt(alpha))

    plt.plot([wm, wm], [-np.pi, np.pi], linestyle='--', color='blue', label='ωm')
    
    # Set custom yticks for phase between pi and -pi
    plt.yticks(np.arange(-np.pi, np.pi + np.pi/2, np.pi/2),
               ['-π', '-π/2', '0', 'π/2', 'π'])
    
    plt.grid(True)
    plt.tight_layout()
    plt.legend()
    plt.show()

# Create sliders for alpha and tau
interact(plot_lead_controller,
         alpha=widgets.FloatSlider(min=0.1, max=1, step=0.1, value=0.0, description='Alpha'),
         tau=widgets.FloatSlider(min=0.1, max=10, step=0.1, value=1.0, description='Tau'))


interactive(children=(FloatSlider(value=0.1, description='Alpha', max=1.0, min=0.1), FloatSlider(value=1.0, de…

<function __main__.plot_lead_controller(alpha, tau)>

## Delay controller effect on closed loop

In [53]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact
import control as ctrl

# Function to plot step response and Bode diagram
def plot_feedback_system(pole1, pole2, zero, gain, alpha, tau):
    # Define the transfer function G(s) = K*(s - zero) / [(s - pole1)*(s - pole2)]
    G = ctrl.TransferFunction([gain, -gain * zero], [1, -(pole1 + pole2), pole1 * pole2])

    # Define the controller R(s) = (1 + s*alpha*tau) / (1 + s*tau)
    R = ctrl.TransferFunction([1, alpha * tau], [1, tau])

    # Calculate the feedback system T(s) = (R*G) / (1 + R*G)
    T = ctrl.feedback(R * G, 1)

    # Time vector for step response
    time, response = ctrl.step_response(T)

    # Frequency range for Bode plot
    freq_range = np.logspace(-2, 2, 1000)
    _, magnitude, phase = signal.bode(signal.TransferFunction(T.num[0][0], T.den[0][0]), freq_range)
    #_, magnitude, phase = signal.bode(signal.TransferFunction(np.array(T.num), np.array(T.den)), freq_range)

    # Plot step response
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 2)
    plt.plot(time, response)
    plt.title('Step Response of closed loop')
    plt.xlabel('Time')
    plt.ylabel('Amplitude')
    plt.grid(True)

    # Plot Bode diagram

    # Magnitude plot
    plt.subplot(2, 2, 1)
    plt.semilogx(freq_range, magnitude)
    plt.title('Bode Diagram of closed loop')
    plt.xlabel('Frequency (rad/s)')
    plt.ylabel('Magnitude (dB)')
    plt.ylim(-30, 10)
    plt.grid(True)

    # Phase plot
    plt.subplot(2, 2, 3)
    plt.semilogx(freq_range, phase * np.pi/180)
    plt.xlabel('Frequency (rad/s)')
    plt.ylabel('Phase (rad)')
    
    # Set custom yticks for phase between pi and -pi
    plt.yticks(np.arange(-np.pi, np.pi + np.pi/2, np.pi/2),
               ['-π', '-π/2', '0', 'π/2', 'π'])
    
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    # Pole-zero map
    plt.figure(figsize=(12, 4))
    poles = G.pole()
    zeros = G.zero()
    plt.scatter(np.real(poles), np.imag(poles), marker='x', color='red', label='Poles')
    plt.scatter(np.real(zeros), np.imag(zeros), marker='o', color='blue', label='Zeros')
    plt.title('Pole-Zero Map of G(s)')
    plt.xlabel('Real')
    plt.ylabel('Imaginary')
    plt.axhline(0, color='black',linewidth=0.5)
    plt.axvline(0, color='black',linewidth=0.5)
    plt.grid(True)
    plt.legend()

# Create sliders for poles, zero, gain, alpha, and tau
pole1_slider = widgets.FloatSlider(min=-5, max=5, step=0.1, value=-1, description='Pole 1:')
pole2_slider = widgets.FloatSlider(min=-5, max=5, step=0.1, value=-2, description='Pole 2:')
zero_slider = widgets.FloatSlider(min=-5, max=5, step=0.1, value=1, description='Zero:')
gain_slider = widgets.FloatSlider(min=0.1, max=10, step=0.1, value=1, description='Gain:')
alpha_slider = widgets.FloatSlider(min=0, max=1, step=0.1, value=1, description='Alpha:')
tau_slider = widgets.FloatSlider(min=0.1, max=10, step=0.1, value=1, description='Tau:')

# Create interactive plot
interact(plot_feedback_system,
         pole1=pole1_slider, pole2=pole2_slider,
         zero=zero_slider, gain=gain_slider,
         alpha=alpha_slider, tau=tau_slider)


interactive(children=(FloatSlider(value=-1.0, description='Pole 1:', max=5.0, min=-5.0), FloatSlider(value=-2.…

<function __main__.plot_feedback_system(pole1, pole2, zero, gain, alpha, tau)>